In [1]:
import scipy as sp
from scipy import signal
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual
import numpy as np

%matplotlib ipympl


# Mono tone

In [2]:
@interact_manual(wm=(0, 10), wc=(0, 100), am=(0, 5, 0.1), ac=(0, 5, 0.1), sampling_freq=(0, 1000))
def amplitude_mod(wm=2, wc=20, am=2, ac=2, sampling_freq=200, mod_type=["simple", "DSBSC", "SSBSC"]):
    plt.close()
    x = np.arange(1, 4, 1/sampling_freq)
    baseband = am*np.sin(2*np.pi*wm*x)
    carrier = ac*np.sin(2*np.pi*wc*x)
    ffx = np.fft.rfftfreq(len(x), 1/sampling_freq)
    bb_ft = np.fft.rfft(baseband)/len(baseband)
    c_ft = np.fft.rfft(carrier)/len(baseband)
    ft_x_scale = int(120*len(x)/sampling_freq)
    ft_x_ticks = np.arange(min(ffx), 120, 5)
    
    
    fig, ax = plt.subplots(3, 2)
    ax[0][0].plot(x, baseband)
    ax[0][0].set_title("Baseband Signal")
    ax[0][1].plot(ffx[:ft_x_scale], 2*np.absolute(bb_ft)[:ft_x_scale])
    ax[0][1].set_xticks(ft_x_ticks)
    ax[0][1].set_title("Baseband Signal Fourier Transform")

    ax[1][0].plot(x, carrier)
    ax[1][0].set_title("Carrier Signal")
    ax[1][1].plot(ffx[:ft_x_scale], 2*np.absolute(c_ft)[:ft_x_scale])
    ax[1][1].set_xticks(ft_x_ticks)
    ax[1][1].set_title("Carrier Signal Fourier Transform")


    if mod_type == "simple":
        modulated_signal = (ac + baseband)*np.sin(2*np.pi*wc*x)
    elif mod_type == "DSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
    elif mod_type == "SSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
        sos = signal.butter(50, wc, 'highpass', output='sos', fs=sampling_freq)
        modulated_signal = signal.sosfilt(sos, modulated_signal)

    mod_ft = np.fft.rfft(modulated_signal)/len(modulated_signal)

    ax[2][0].plot(x, modulated_signal)
    ax[2][0].set_title("Modulated Signal")
    ax[2][1].plot(ffx[:ft_x_scale], 2*np.absolute(mod_ft)[:ft_x_scale])
    ax[2][1].set_xticks(ft_x_ticks)
    ax[2][1].set_title("Modulated Signal Fourier Transform")

    
    fig.set_figwidth(15)
    fig.set_figheight(6)
    fig.tight_layout(pad=1.0)

    print("Modulating Index : ", am/ac)
    pc = 2*sum(np.absolute(c_ft)**2)
    pam = 2*sum(np.absolute(mod_ft)**2)
    print("Pc = %.3f"%(pc))
    print("Pam = %.3f"%(pam))



interactive(children=(IntSlider(value=2, description='wm', max=10), IntSlider(value=20, description='wc'), Flo…

# Two tone

In [3]:
@interact_manual(wm1=(0, 10), wm2=(0, 10), wc=(0, 100),am1=(0, 5, 0.1), am2=(0, 5, 0.1), ac=(0, 5, 0.1), sampling_freq=(0, 1000))
def two_tone_mod(wm1=3, wm2=4, wc=50, am1=1, am2=3, ac=5, sampling_freq=510, mod_type=["simple", "DSBSC", "SSBSC"]):
    plt.close()
    x = np.arange(1, 4, 1/sampling_freq)
    signal1 = am1*np.sin(2*np.pi*wm1*x)
    signal2 = am2*np.sin(2*np.pi*wm2*x)
    baseband = signal1 + signal2
    carrier = ac*np.sin(2*np.pi*wc*x)

    ffx = np.fft.rfftfreq(len(x), 1/sampling_freq)
    bb_ft = np.fft.rfft(baseband)/len(baseband)
    c_ft = np.fft.rfft(carrier)/len(carrier)
    s1_ft = np.fft.rfft(signal1)/len(signal1)
    s2_ft = np.fft.rfft(signal2)/len(signal2)

    ft_x_scale = int(120*len(x)/sampling_freq)
    ft_x_ticks = np.arange(min(ffx), 120, 5)
    
    
    fig, ax = plt.subplots(5, 2)
    ax[0][0].plot(x, signal1)
    ax[0][1].plot(ffx[:ft_x_scale//10], 2*np.absolute(s1_ft)[:ft_x_scale//10])
    ax[0][1].set_title("Signal 1")
    ax[0][1].set_xticks(ft_x_ticks//10)
    ax[1][0].plot(x, signal2)
    ax[1][1].plot(ffx[:ft_x_scale//10], 2*np.absolute(s2_ft)[:ft_x_scale//10])
    ax[1][1].set_title("Signal 2")
    ax[1][1].set_xticks(ft_x_ticks//10)


    ax[2][0].plot(x, baseband)
    ax[2][0].set_title("Baseband Signal")
    ax[2][1].plot(ffx[:ft_x_scale//10], 2*np.absolute(bb_ft)[:ft_x_scale//10])
    ax[2][1].set_xticks(ft_x_ticks//10)
    ax[2][1].set_title("Baseband Signal Fourier Transform")

    ax[3][0].plot(x, carrier)
    ax[3][0].set_title("Carrier Signal")
    ax[3][1].plot(ffx[:ft_x_scale], 2*np.absolute(c_ft)[:ft_x_scale])
    ax[3][1].set_xticks(ft_x_ticks)
    ax[3][1].set_title("Carrier Signal Fourier Transform")

    if mod_type == "simple":
        modulated_signal = (ac + baseband)*np.sin(2*np.pi*wc*x)
    elif mod_type == "DSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
    elif mod_type == "SSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
        sos = signal.butter(30,wc, 'hp', output='sos', fs=sampling_freq)
        modulated_signal = signal.sosfilt(sos, modulated_signal)

    mod_ft = np.fft.rfft(modulated_signal)/len(modulated_signal)

    ax[4][0].plot(x, modulated_signal)
    ax[4][0].set_title("Modulated Signal")
    ax[4][1].plot(ffx[:ft_x_scale], 2*np.absolute(mod_ft)[:ft_x_scale])
    ax[4][1].set_xticks(ft_x_ticks)
    ax[4][1].set_title("Modulated Signal Fourier Transform")

    
    fig.set_figwidth(15)
    fig.set_figheight(12)
    fig.tight_layout(pad=1)

    print("Modulating Index 1 : ", am1/ac)
    print("Modulating Index 2 : ", am2/ac)
    print("M1 + M2 = ", (am1 + am2)/ac)
    pc = 2*sum(np.absolute(c_ft)**2)
    pam = 2*sum(np.absolute(mod_ft)**2)
    print("Pc = %.3f"%(pc))
    print("Pam = %.3f"%(pam))


interactive(children=(IntSlider(value=3, description='wm1', max=10), IntSlider(value=4, description='wm2', max…

# Multi tone 

In [7]:
def multi_tone_mod(wm=[], am=[], wc=50, ac=2, sampling_freq=200, mod_type="simple"):
    plt.close()
    x = np.arange(1, 4, 1/sampling_freq)
    
    if len(wm) != len(am):
        raise "wm and am are not of the same dimension"

    baseband = np.zeros(len(x))
    for i in range(len(wm)):
        baseband += am[i]*np.sin(2*np.pi*wm[i]*x)
    


    carrier = ac*np.sin(2*np.pi*wc*x)
    ffx = np.fft.rfftfreq(len(x), 1/sampling_freq)
    bb_ft = np.fft.rfft(baseband)/len(baseband)
    c_ft = np.fft.rfft(carrier)/len(baseband)
    ft_x_scale = int(120*len(x)/sampling_freq)
    ft_x_ticks = np.arange(min(ffx), 120, 5)
    
    
    fig, ax = plt.subplots(3, 2)
    ax[0][0].plot(x, baseband)
    ax[0][0].set_title("Baseband Signal")
    ax[0][1].stem(ffx[:ft_x_scale//10], 2*np.absolute(bb_ft)[:ft_x_scale//10])
    ax[0][1].set_xticks(ft_x_ticks//10)
    ax[0][1].set_title("Baseband Signal Fourier Transform")

    ax[1][0].plot(x, carrier)
    ax[1][0].set_title("Carrier Signal")
    ax[1][1].stem(ffx[:ft_x_scale], 2*np.absolute(c_ft)[:ft_x_scale])
    ax[1][1].set_xticks(ft_x_ticks)
    ax[1][1].set_title("Carrier Signal Fourier Transform")


    if mod_type == "simple":
        modulated_signal = (ac + baseband)*np.sin(2*np.pi*wc*x)
    elif mod_type == "DSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
    elif mod_type == "SSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
        sos = signal.butter(50, wc, 'highpass', output='sos', fs=sampling_freq)
        modulated_signal = signal.sosfilt(sos, modulated_signal)

    mod_ft = np.fft.rfft(modulated_signal)/len(modulated_signal)

    ax[2][0].plot(x, modulated_signal)
    ax[2][0].set_title("Modulated Signal")
    ax[2][1].plot(ffx[:ft_x_scale], 2*np.absolute(mod_ft)[:ft_x_scale])
    ax[2][1].set_xticks(ft_x_ticks)
    ax[2][1].set_title("Modulated Signal Fourier Transform")

    
    fig.set_figwidth(16)
    fig.set_figheight(8)
    fig.tight_layout(pad=5.0)

    print("Modulating Index : ", sum(am)/ac)
    pc = 2*sum(np.absolute(c_ft)**2)
    pam = 2*sum(np.absolute(mod_ft)**2)
    # print("Pc = %.3f"%(pc))
    # print("Pam = %.3f"%(pam))

In [8]:
wm = [1, 1.5, 2, 3, 4.4, 4, 5, 5.7]
am = [.2, .1, 1.3, .2, .3, 0.4, .2, 1.8]
sampling_frequency = 500
@interact_manual
def multi_tone_mod_interact(wc=(0, 100, 1), ac=(0, 5, 0.1), mod_type=["simple", "DSBSC", "SSBSC"]):
    multi_tone_mod(wm, am, wc, ac, sampling_frequency ,mod_type)

interactive(children=(IntSlider(value=50, description='wc'), FloatSlider(value=2.0, description='ac', max=5.0)…

# Any baseband signal

In [4]:
def any_baseband(x, baseband, wc=50, ac=2, sampling_freq=200, mod_type="simple"):
    plt.close()
    
    carrier = ac*np.sin(2*np.pi*wc*x)
    ffx = np.fft.fftfreq(len(x), 1/sampling_freq)
    bb_ft = np.fft.fft(baseband)/len(baseband)
    c_ft = np.fft.fft(carrier)/len(baseband)
    
    
    fig, ax = plt.subplots(3, 2)
    ax[0][0].plot(x, baseband)
    ax[0][0].set_title("Baseband Signal")
    ax[0][1].plot(ffx, np.absolute(bb_ft))
    ax[0][1].set_title("Baseband Signal Fourier Transform")

    ax[1][0].plot(x, carrier)
    ax[1][0].set_title("Carrier Signal")
    ax[1][1].plot(ffx, np.absolute(c_ft))
    ax[1][1].set_title("Carrier Signal Fourier Transform")


    if mod_type == "simple":
        modulated_signal = (ac + baseband)*np.sin(2*np.pi*wc*x)
    elif mod_type == "DSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
    elif mod_type == "SSBSC":
        modulated_signal = baseband*np.sin(2*np.pi*wc*x)
        sos = signal.butter(50, 'highpass', output='sos', fs=sampling_freq)
        modulated_signal = signal.sosfilt(sos, modulated_signal)

    mod_ft = np.fft.fft(modulated_signal)/len(modulated_signal)

    ax[2][0].plot(x, modulated_signal)
    ax[2][0].set_title("Modulated Signal")
    ax[2][1].plot(ffx, np.absolute(mod_ft))
    ax[2][1].set_title("Modulated Signal Fourier Transform")

    
    fig.set_figwidth(16)
    fig.set_figheight(8)
    fig.tight_layout(pad=5.0)


In [5]:
sampling_frequency = 500
x = np.arange(-4, 4, 1/sampling_frequency)
baseband_signal = np.sinc(10*x)**2

@interact_manual(wm=(0, 10), wc=(0, 100), am=(0, 5, 0.1), ac=(0, 5, 0.1))
def any_baseband_interact(wc=20, ac=2, mod_type=["simple", "DSBSC"]): 
    any_baseband(x, baseband_signal, wc, ac, sampling_frequency, mod_type)
    

interactive(children=(IntSlider(value=20, description='wc'), FloatSlider(value=2.0, description='ac', max=5.0)…

# Demodulation

In [3]:
def any_baseband(x, baseband, fc=50, ac=2, sampling_freq=200, lpf_freq=10):
    plt.close()
    
    carrier = ac*np.cos(2*np.pi*fc*x)
    ffx = np.fft.fftfreq(len(x), 1/sampling_freq)
    bb_ft = np.fft.fft(baseband)/len(baseband)
    c_ft = np.fft.fft(carrier)/len(baseband)
    
    
    fig, ax = plt.subplots(5, 2)
    ax[0][0].plot(x, baseband)
    ax[0][0].set_title("Baseband Signal")
    ax[0][1].plot(ffx, np.absolute(bb_ft))
    ax[0][1].set_title("Baseband Signal Fourier Transform")

    ax[1][0].plot(x, carrier)
    ax[1][0].set_title("Carrier Signal")
    ax[1][1].plot(ffx, np.absolute(c_ft))
    ax[1][1].set_title("Carrier Signal Fourier Transform")



    modulated_signal = baseband*carrier


    mod_ft = np.fft.fft(modulated_signal)/len(modulated_signal)

    ax[2][0].plot(x, modulated_signal)
    ax[2][0].set_title("Modulated Signal")
    ax[2][1].plot(ffx, np.absolute(mod_ft))
    ax[2][1].set_title("Modulated Signal Fourier Transform")

    intermediate_signal = modulated_signal*np.cos(2*np.pi*fc*x)
    i_ft = np.fft.fft(intermediate_signal)/len(intermediate_signal)

    ax[3][0].plot(x, intermediate_signal)
    ax[3][0].set_title("Intermediate Signal")
    ax[3][1].plot(ffx, np.absolute(i_ft))
    ax[3][1].set_title("Intermediate Signal Fourier Transform")
    
    lpf = []
    for xi in ffx:
        if np.abs(xi) <= lpf_freq:
            lpf.append(1)
        else:
            lpf.append(0)
    
    lpf = np.array(lpf)
    demod_ft = i_ft*lpf
    demod_sig = np.fft.ifft(demod_ft)*len(demod_ft)

    ax[4][0].plot(x, demod_sig)
    ax[4][1].plot(ffx, np.abs(demod_ft))

    
    fig.set_figwidth(16)
    fig.set_figheight(20)
    fig.tight_layout(pad=5.0)

sampling_frequency = 500
x = np.arange(-4, 4, 1/sampling_frequency)
baseband_signal = np.sinc(30*x)*2

@interact_manual()
def any_baseband_interact(fc=(0, 100, 1), ac=(0, 5, 1), lpf_freq=(0, 20, 1)): 
    any_baseband(x, baseband_signal, fc, ac, sampling_frequency, lpf_freq)

interactive(children=(IntSlider(value=50, description='fc'), IntSlider(value=2, description='ac', max=5), IntS…

In [14]:
x = np.arange(0, 3, 1/100)
sig = 3*np.sin(2*np.pi*4*x) + 2*np.sin(2*np.pi*3*x) + 5*np.sin(2*np.pi*5*x)
plt.close()
plt.plot(x, sig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
ft = np.fft.fft(sig)
plt.close()
plt.plot(np.abs(ft))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
ft_s = np.fft.fftshift(ft)
plt.close()
plt.plot(np.abs(ft_s))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
ftx = np.fft.fftfreq(len(x), 1/100)
plt.close()
plt.plot(ftx, abs(ft))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
ftx_s = np.fft.fftshift(ftx)
plt.close()
plt.plot(ftx_s, np.abs(ft_s))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …